In [1]:
from cv2 import aruco
import cv2
import numpy as np
import os
import csv
import sys
from tqdm import tqdm

In [2]:
_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
_pth = os.path.join(_pth, 'ArUco_detection', "images")
image_list = os.listdir(_pth)

In [3]:
ARUCO_PARAMETERS = aruco.DetectorParameters()
DICTIONARY = aruco.getPredefinedDictionary(aruco.DICT_ARUCO_ORIGINAL)
detector = aruco.ArucoDetector(DICTIONARY, ARUCO_PARAMETERS)

In [4]:
raw_data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
raw_data_pth = os.path.join(raw_data_pth, 'dataset', "raw_data")
if not os.path.exists(raw_data_pth):
    os.makedirs(os.path.join(raw_data_pth, "images"))
    os.makedirs(os.path.join(raw_data_pth, "labels"))

In [5]:
for img_name in tqdm(image_list):
    _image_path = os.path.join(_pth, img_name)
    image = cv2.imread(_image_path)
    markerCorners, markerIds, rejectedCandidates = detector.detectMarkers(image)
    
    if len(markerCorners) == 0:
        continue
    
    label_name = img_name.split(".")[0]
    label_path = os.path.join(raw_data_pth, "labels", f"{label_name}.txt")
    label_file = open(label_path, "w")
    label_writer = csv.writer(label_file, delimiter=" ")
    _markerCorners = markerCorners[0][0]
    label_writer.writerow(["0", _markerCorners[0][0], _markerCorners[0][1], _markerCorners[1][0], _markerCorners[1][1], _markerCorners[2][0], _markerCorners[2][1], _markerCorners[3][0], _markerCorners[3][1], _markerCorners[0][0], _markerCorners[0][1]])
    label_file.close()
    
    # save image
    image_path = os.path.join(raw_data_pth, "images", img_name)
    cv2.imwrite(image_path, image)


100%|██████████| 1963/1963 [03:31<00:00,  9.29it/s]


In [6]:
print(len(os.listdir(os.path.join(raw_data_pth, "labels"))), len(image_list))

1835 1963


Splitting it into traning and testing dataset

In [10]:
# splitting dataset into train and validation and test
data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_pth = os.path.join(data_pth, 'dataset', "dataset_processed")

train_data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
train_data_pth = os.path.join(train_data_pth, 'dataset_processed', "train")
validation_data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
validation_data_pth = os.path.join(validation_data_pth, 'dataset_processed', "val")
test_data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
test_data_pth = os.path.join(test_data_pth, 'dataset_processed', "test")

if not os.path.exists(train_data_pth):
    os.makedirs(os.path.join(train_data_pth, "images"))
    os.makedirs(os.path.join(train_data_pth, "labels"))

if not os.path.exists(test_data_pth):
    os.makedirs(os.path.join(test_data_pth, "images"))
    os.makedirs(os.path.join(test_data_pth, "labels"))
    
if not os.path.exists(validation_data_pth):
    os.makedirs(os.path.join(validation_data_pth, "images"))
    os.makedirs(os.path.join(validation_data_pth, "labels"))
    
image_list = os.listdir(os.path.join(raw_data_pth, "images"))

# splitting dataset into train and test
for img_name in tqdm(image_list):
    
    _image_path = os.path.join(raw_data_pth, "images", img_name)
    image = cv2.imread(_image_path)
    
    label_name = img_name.split(".")[0]
    label_path = os.path.join(raw_data_pth, "labels", f"{label_name}.txt")
    label_file = open(label_path, "r")
    label_reader = csv.reader(label_file, delimiter=" ")
    label = list(label_reader)[0]
    label_file.close()
    
    if int(label[0]) == 0:
        if np.random.rand() < 0.7:
            # save image
            image_path = os.path.join(train_data_pth, "images", img_name)
            cv2.imwrite(image_path, image)
            
            label_path = os.path.join(train_data_pth, "labels", f"{label_name}.txt")
            label_file = open(label_path, "w")
            label_writer = csv.writer(label_file, delimiter=" ")
            label_writer.writerow(label)
            label_file.close()
            
        elif np.random.rand() < 0.9 and np.random.rand() > 0.7:
            # save image
            image_path = os.path.join(validation_data_pth, "images", img_name)
            cv2.imwrite(image_path, image)
            
            label_path = os.path.join(validation_data_pth, "labels", f"{label_name}.txt")
            label_file = open(label_path, "w")
            label_writer = csv.writer(label_file, delimiter=" ")
            label_writer.writerow(label)
            label_file.close()
            
        else:
            # save image
            image_path = os.path.join(test_data_pth, "images", img_name)
            cv2.imwrite(image_path, image)
            
            label_path = os.path.join(test_data_pth, "labels", f"{label_name}.txt")
            label_file = open(label_path, "w")
            label_writer = csv.writer(label_file, delimiter=" ")
            label_writer.writerow(label)
            label_file.close() 
    


100%|██████████| 1835/1835 [02:21<00:00, 12.98it/s]


In [11]:
# splitting dataset into train and validation and test
data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_pth = os.path.join(data_pth, 'dataset', "dataset_processed")

images_pth = os.path.join(data_pth, "images")
labels_pth = os.path.join(data_pth, "labels")

if not os.path.exists(images_pth):
    os.makedirs(os.path.join(images_pth, "train"))
    os.makedirs(os.path.join(images_pth, "val"))
    os.makedirs(os.path.join(images_pth, "test"))

if not os.path.exists(labels_pth):
    os.makedirs(os.path.join(labels_pth, "train"))
    os.makedirs(os.path.join(labels_pth, "val"))
    os.makedirs(os.path.join(labels_pth, "test"))
    
    
image_list = os.listdir(os.path.join(raw_data_pth, "images"))

# splitting dataset into train and test
for img_name in tqdm(image_list):
    
    _image_path = os.path.join(raw_data_pth, "images", img_name)
    image = cv2.imread(_image_path)
    
    label_name = img_name.split(".")[0]
    label_path = os.path.join(raw_data_pth, "labels", f"{label_name}.txt")
    label_file = open(label_path, "r")
    label_reader = csv.reader(label_file, delimiter=" ")
    label = list(label_reader)[0]
    label_file.close()
    
    if int(label[0]) == 0:
        if np.random.rand() < 0.7:
            # save image
            image_path = os.path.join(os.path.join(images_pth, "train"), img_name)
            cv2.imwrite(image_path, image)
            
            label_path = os.path.join(os.path.join(labels_pth, "train"), f"{label_name}.txt")
            label_file = open(label_path, "w")
            label_writer = csv.writer(label_file, delimiter=" ")
            label_writer.writerow(label)
            label_file.close()
            
        elif np.random.rand() < 0.9 and np.random.rand() > 0.7:
            # save image
            image_path = os.path.join(os.path.join(images_pth, "val"), img_name)
            cv2.imwrite(image_path, image)
            
            label_path = os.path.join(os.path.join(labels_pth, "val"), f"{label_name}.txt")
            label_file = open(label_path, "w")
            label_writer = csv.writer(label_file, delimiter=" ")
            label_writer.writerow(label)
            label_file.close()
            
        else:
            # save image
            image_path = os.path.join(os.path.join(images_pth, "test"), img_name)
            cv2.imwrite(image_path, image)
            
            label_path = os.path.join(os.path.join(labels_pth, "test"), f"{label_name}.txt")
            label_file = open(label_path, "w")
            label_writer = csv.writer(label_file, delimiter=" ")
            label_writer.writerow(label)
            label_file.close() 


100%|██████████| 1835/1835 [02:39<00:00, 11.49it/s]
